In [ ]:
import tensorflow as tf
print("tensorflow version " + str(tf.__version__)) # tested on 2.4.1
import tensorflow_addons as tfa
print("tensorflow-addons version " + str(tfa.__version__))
from tensorboard import notebook
%load_ext tensorboard

from datetime import datetime
import numpy as np
#from neuraltaa.dense_image_warp_seqtest import *
from dense_image_warp import *

import matplotlib.pyplot as plt
plt.style.use("dark_background")


In [ ]:
w=128
h=128
scale = 10
batch_size=8*4
imgs=tf.random.poisson((batch_size,w,h,3),lam=0.5)

In [ ]:
### zooms on the top left corner ###

x = tf.linspace(0.0,1.0/scale*(w-1),w)
y = tf.linspace(0.0,1.0/scale*(h-1),h)
grid_x, grid_y= tf.meshgrid(x,y)
stacked_grid = tf.cast(tf.stack([grid_y, grid_x], axis=2), float)
batched_grid = tf.broadcast_to(stacked_grid,(batch_size,)+stacked_grid.shape)
query_points_on_grid = batched_grid
print(tf.shape(batched_grid))
query_points_flattened = tf.reshape(
    query_points_on_grid, [batch_size, h * w, 2]
)
print(tf.shape(query_points_flattened))

In [ ]:
# Catmull
tf.profiler.experimental.start('logtest/catmull')
res=interpolate_catmull_rom(imgs,query_points_flattened)
tf.profiler.experimental.stop()

In [ ]:
# Bilinear
tf.profiler.experimental.start('logtest/bilin')
res2=interpolate_bilinear(imgs,query_points_flattened)
tf.profiler.experimental.stop()

In [ ]:
print(np.shape(res))
print(np.shape(res2))

In [ ]:
res=tf.reshape(res, [batch_size, h, w, 3])
res2=tf.reshape(res2, [batch_size, h, w, 3])
plt.imshow(imgs[0])
plt.figure()

fig, axs = plt.subplots(1,2,figsize=(13,7))
fig.suptitle('left: Catmull-Rom, right: bilinear\nZoom x'+str(scale), fontsize=35)
axs[0].imshow(res[0])
axs[1].imshow(res2[0])


notice the cross like behavior of bilinear interpolation,
this artifact can be problematic to network training or create artifacts in fourier reconstructions